In [2]:
%matplotlib inline

import warnings
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

import random
import pandas as pd

pd.options.mode.use_inf_as_na = True
# warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
NUM_TEST = 100
NUM_ITERS=10000
DEFAULT_TOL = 1e-12

In [4]:
likelihood_vect = np.empty([0,1])
aic_vect = np.empty([0,1])
bic_vect = np.empty([0,1])

# Possible number of states in Markov Model
STATE_SPACE = range(2,15)

In [5]:
csv_data = pd.read_csv('SP500.csv')

Date = csv_data['Date']
Close = csv_data['Close']

In [6]:
# clean up data
# delete extra data (Date[0],Adj Close[4], Volume[5]) and leave Open/Hight/Low/Close
del(csv_data['Date'])
del(csv_data['Adj Close'])
del(csv_data['Volume'])

csv_data.head()

,Open,High,Low,Close
0,16.66,16.66,16.66,16.66
1,16.85,16.85,16.85,16.85
2,16.93,16.93,16.93,16.93
3,16.98,16.98,16.98,16.98
4,17.08,17.08,17.08,17.08


In [7]:
# csv_data['Volume'] = (csv_data['Volume'] / 1000).astype(int) / 100
# csv_data.head()

In [8]:
def get_exp_preprocessing(df,alpha=0.9):
    edata = df.ewm(alpha=alpha).mean()
    return edata

In [9]:
dataset = get_exp_preprocessing(csv_data)
dataset.head()

,Open,High,Low,Close
0,16.660000,16.660000,16.660000,16.660000
1,16.832727,16.832727,16.832727,16.832727
2,16.920360,16.920360,16.920360,16.920360
3,16.974041,16.974041,16.974041,16.974041
4,17.069405,17.069405,17.069405,17.069405


In [10]:
dataset.shape

(17320, 4)

In [11]:
# dataset = (dataset*100).astype(int)/100
# dataset[:,-1] = dataset[:,-1] / 1000

In [12]:
predicted_stock_data = np.empty([0,dataset.shape[1]])
likelihood_vect = np.empty([0,1])
aic_vect = np.empty([0,1])
bic_vect = np.empty([0,1])

In [13]:
dataset = np.array(dataset)

In [14]:
# TEST START 1

In [15]:
states = 10

num_params = states**2 + states
dirichlet_params_states = np.random.randint(1,50,states)
#model = hmm.GaussianHMM(n_components=states, covariance_type='full', startprob_prior=dirichlet_params_states, transmat_prior=dirichlet_params_states, tol=0.0001, n_iter=NUM_ITERS, init_params='mc')
model = hmm.GaussianHMM(n_components=states, covariance_type='full', tol=DEFAULT_TOL, n_iter=NUM_ITERS)
# model.fit(dataset[1:,:])
model.fit(dataset)

# if model.monitor_.iter == NUM_ITERS:
#     print('Increase number of iterations')
#     sys.exit(1)
# likelihood_vect = np.vstack((likelihood_vect, model.score(dataset)))

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=10, n_iter=10000, params='stmc',
      random_state=None, startprob_prior=1.0, tol=1e-12,
      transmat_prior=1.0, verbose=False)

In [16]:
state_sequence = model.predict(dataset)

In [17]:
state_sequence

array([5, 5, 5, ..., 4, 4, 4])

In [ ]:
plt.figure(figsize=(25,18))
for i in range(model.n_components):
    pos = (state_sequence == i)
    plt.plot_date(Date[pos],Close[pos],'o',label='hidden state %d'%i,lw=2)
    plt.legend(loc='upper left')

In [23]:
state_sum = {}
state_meaning = {}
state_num_negative = {}
state_num_postive = {}
for i in range(len(state_sequence)-1):
    x = state_sequence[i]
    state_sum[x] = state_sum.get(x, 0) + 1
    dif_change = (dataset[i+1][1] - dataset[i][1])
    state_meaning[x] = state_sum.get(x, 0) + dif_change
    if dif_change > 0:
        state_num_postive[x] = state_num_postive.get(x, 0) + 1
    else:
        state_num_negative[x] = state_num_negative.get(x, 0) + 1


In [24]:
state_num_negative

{0: 1333,
 1: 582,
 2: 332,
 3: 105,
 4: 1557,
 5: 692,
 6: 428,
 7: 2409,
 8: 256,
 9: 305}

In [25]:
state_num_postive

{0: 1700,
 1: 502,
 2: 480,
 3: 96,
 4: 1863,
 5: 374,
 6: 515,
 7: 2593,
 8: 254,
 9: 943}

In [27]:
state_change = {}
for i in range(len(state_meaning)):
    state_change[i] = {"avg":state_meaning[i] / state_sum[i], 
                       "postive":state_num_postive[i]/state_sum[i], 
                       "negative":state_num_negative[i]/state_sum[i]}

for k in state_change:
    print(k, state_change[k])

0 {'avg': 1.000233549941793, 'postive': 0.5605011539729641, 'negative': 0.4394988460270359}
1 {'avg': 0.9995593395851032, 'postive': 0.46309963099630996, 'negative': 0.5369003690036901}
2 {'avg': 1.008905188764099, 'postive': 0.5911330049261084, 'negative': 0.4088669950738916}
3 {'avg': 0.9046146152625372, 'postive': 0.4752475247524752, 'negative': 0.5198019801980198}
4 {'avg': 1.000288506528792, 'postive': 0.5447368421052632, 'negative': 0.45526315789473687}
5 {'avg': 1.0009012004240982, 'postive': 0.350844277673546, 'negative': 0.649155722326454}
6 {'avg': 1.0017902179928495, 'postive': 0.5461293743372216, 'negative': 0.45387062566277836}
7 {'avg': 1.0005966732832923, 'postive': 0.5183926429428228, 'negative': 0.4816073570571771}
8 {'avg': 0.9517781044342446, 'postive': 0.4980392156862745, 'negative': 0.5019607843137255}
9 {'avg': 1.0023846622968915, 'postive': 0.7556089743589743, 'negative': 0.24439102564102563}


In [26]:
state_sum = {}
for x in state_sequence:
    state_sum[x] = state_sum.get(x, 0) + 1
    
print(state_sum)

{0: 3033, 7: 5002, 4: 3420, 6: 943, 3: 202, 1: 1084, 9: 1248, 5: 1066, 2: 812, 8: 510}


In [49]:
model.predict_proba(dataset)[17318]

array([0.00000000e+000, 2.04922584e-034, 6.89911637e-149, 1.00000000e+000,
       0.00000000e+000, 5.67921793e-129, 3.75201657e-073, 0.00000000e+000,
       9.68063352e-021, 0.00000000e+000])

In [66]:
model.transmat_.shape

(10, 10)

In [67]:
model.transmat_

array([[9.99011532e-001, 4.87742615e-305, 0.00000000e+000,
        4.92676431e-237, 3.00244464e-053, 0.00000000e+000,
        1.79027489e-130, 9.88467876e-004, 0.00000000e+000,
        0.00000000e+000],
       [2.68244640e-283, 8.28692730e-001, 8.10289771e-023,
        5.52544204e-002, 3.81105353e-070, 3.06045218e-002,
        1.09322097e-002, 2.06665250e-180, 3.21963912e-030,
        7.45161176e-002],
       [0.00000000e+000, 3.20895136e-050, 8.56291636e-001,
        1.77441729e-030, 5.47346924e-181, 8.61835722e-046,
        2.10830864e-092, 0.00000000e+000, 1.43708364e-001,
        1.53217292e-049],
       [8.02442735e-214, 2.75155684e-001, 1.05417548e-025,
        6.50053462e-001, 6.03159868e-036, 5.41760122e-007,
        6.84369171e-003, 8.64408823e-127, 6.79466209e-002,
        2.66459846e-010],
       [3.89355645e-050, 4.41992613e-054, 2.75701995e-181,
        2.06927342e-023, 9.76270692e-001, 4.03903355e-122,
        1.25445997e-002, 1.11847088e-002, 2.22729668e-108,
        3.0

In [0]:
prob_next_step = model.transmat_[state_sequence[-1], :]

In [0]:
prob_next_step

array([1.74163913e-215, 5.82987900e-002, 2.07399929e-205, 7.41386677e-162,
       1.07778966e-063, 1.79698744e-001, 2.98098836e-001, 6.45560179e-134,
       2.99606810e-001, 1.64296819e-001])

In [63]:
logprob, posteriors = model.score_samples(dataset)

In [65]:
posteriors

(17320, 10)

In [52]:
curr_likelihood = model.score(dataset)

In [53]:
curr_likelihood

-65173.65827326065

In [56]:
iters = 1;
past_likelihood = []
num_examples = dataset.shape[0]
K = 5
while iters < num_examples / K - 1:
    past_likelihood = np.append(past_likelihood, model.score((dataset[iters:iters + K - 1, :])))
    iters = iters + 1

In [57]:
print(num_examples)
print(past_likelihood)
print(curr_likelihood)
print(iters)
print(K)

17320
[ 118.46018291  118.47886072  118.49404198 ... -300.4091593  -299.09981271
 -299.20326895]
-65173.65827326065
3463
5


In [68]:
likelihood_diff_idx = np.argmin(np.absolute(past_likelihood - curr_likelihood))
likelihood_diff_idx

3115

In [69]:
predicted_change = dataset[likelihood_diff_idx,:] - dataset[likelihood_diff_idx + 1,:]
predicted_change

array([ 3.42293325,  0.94667497,  2.39605452, -1.95378856])

In [0]:
# TEST END 1

for states in STATE_SPACE:
    print(states)
    num_params = states**2 + states
    dirichlet_params_states = np.random.randint(1,50,states)
    #model = hmm.GaussianHMM(n_components=states, covariance_type='full', startprob_prior=dirichlet_params_states, transmat_prior=dirichlet_params_states, tol=0.0001, n_iter=NUM_ITERS, init_params='mc')
    model = hmm.GaussianHMM(n_components=states, covariance_type='full', tol=DEFAULT_TOL, n_iter=NUM_ITERS)
    model.fit(dataset[NUM_TEST:,:])
    if model.monitor_.iter == NUM_ITERS:
        print('Increase number of iterations')
        sys.exit(1)
    likelihood_vect = np.vstack((likelihood_vect, model.score(dataset)))
    aic_vect = np.vstack((aic_vect, -2 * model.score(dataset) + 2 * num_params))
    bic_vect = np.vstack((bic_vect, -2 * model.score(dataset) +  num_params * np.log(dataset.shape[0])))

opt_states = np.argmin(bic_vect) + 2
print('Optimum number of states are {}'.format(opt_states))

for idx in reversed(range(NUM_TEST)):
    train_dataset = dataset[idx + 1:,:]
    test_data = dataset[idx,:]; 
    num_examples = train_dataset.shape[0]
    #model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', startprob_prior=dirichlet_params, transmat_prior=dirichlet_params, tol=0.0001, n_iter=NUM_ITERS, init_params='mc')
    if idx == NUM_TEST - 1:
        model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', tol=DEFAULT_TOL, n_iter=NUM_ITERS, init_params='stmc')
    else:
        # Retune the model by using the HMM paramters from the previous iterations as the prior
        model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', tol=DEFAULT_TOL, n_iter=NUM_ITERS, init_params='')
        model.transmat_ = transmat_retune_prior 
        model.startprob_ = startprob_retune_prior
        model.means_ = means_retune_prior
        model.covars_ = covars_retune_prior

    model.fit(np.flipud(train_dataset))

    transmat_retune_prior = model.transmat_
    startprob_retune_prior = model.startprob_
    means_retune_prior = model.means_
    covars_retune_prior = model.covars_

    if model.monitor_.iter == NUM_ITERS:
        print('Increase number of iterations')
        sys.exit(1)
    #print('Model score : ', model.score(dataset))
    #print('Dirichlet parameters : ',dirichlet_params)

    iters = 1;
    past_likelihood = []
    curr_likelihood = model.score(np.flipud(train_dataset[0:K - 1, :]))
    while iters < num_examples / K - 1:
        past_likelihood = np.append(past_likelihood, model.score(np.flipud(train_dataset[iters:iters + K - 1, :])))
        iters = iters + 1
    likelihood_diff_idx = np.argmin(np.absolute(past_likelihood - curr_likelihood))
    predicted_change = train_dataset[likelihood_diff_idx,:] - train_dataset[likelihood_diff_idx + 1,:]
    predicted_stock_data = np.vstack((predicted_stock_data, dataset[idx + 1,:] + predicted_change))
np.savetxt('{}_forecast.csv'.format(stock),predicted_stock_data,delimiter=',',fmt='%.2f')

def calc_mape(predicted_data, true_data):
    return np.divide(np.sum(np.divide(np.absolute(predicted_data - true_data), true_data), 0), true_data.shape[0])

labels = ['Open','High','Low','Close','Volume']
PLOT_TYPE = False

mape = calc_mape(predicted_stock_data, np.flipud(dataset[range(100),:]))
print('MAPE for the stock {} is '.format(stock),mape)

if PLOT_TYPE:
    hdl_p = plt.plot(range(100), predicted_stock_data);
    plt.title('Predicted stock prices')
    plt.legend(iter(hdl_p), ('Open','High','Low','Close','Volume'))
    plt.xlabel('Time steps')
    plt.ylabel('Price')
    plt.figure()
    hdl_a = plt.plot(range(100),np.flipud(dataset[range(100),:]))
    plt.title('Actual stock prices')
    plt.legend(iter(hdl_p), ('Open','High','Low','Close','Volume'))
    plt.xlabel('Time steps')
    plt.ylabel('Price')
else:
    for i in range(len(labels)):
        plt.figure()
        plt.plot(range(100), predicted_stock_data[:,i],'k-', label = 'Predicted '+labels[i]+' price');
        plt.plot(range(100),np.flipud(dataset[range(100),i]),'r--', label = 'Actual '+labels[i]+' price')
        plt.xlabel('Time steps')
        plt.ylabel('Price')
        plt.title(labels[i]+' price'+ ' for '+stock[:-4])
        plt.grid(True)
        plt.legend(loc = 'upper left')


plt.show(block=False)